## Using SMIRNOFF with Amber on BRD4:inhibitor complexes: Exporting parameterized complexes to Amber, Gromacs, and CHARMM

This example applies SMIRNOFF-format parameters to BRD4 inhibitors from the [living review on binding free energy benchmark systems](https://www.annualreviews.org/doi/abs/10.1146/annurev-biophys-070816-033654) by Mobley and Gilson. The BRD4 system comes from the [accompanying GitHub repository](https://github.com/MobleyLab/benchmarksets/tree/master/input_files/BRD4).

In [ ]:
# Retrieve protein and ligand files for BRD4 and a docked inhibitor from the benchmark systems GitHub repository
# https://github.com/MobleyLab/benchmarksets
import requests

repo_url = (
    "https://raw.githubusercontent.com/MobleyLab/benchmarksets/master/input_files/"
)
sources = {
    "receptor.pdb": repo_url + "BRD4/pdb/BRD4.pdb",
    "ligand.pdb": repo_url + "BRD4/pdb/ligand-1.pdb",
    "ligand.sdf": repo_url + "BRD4/sdf/ligand-1.sdf",
}
for filename, url in sources.items():
    r = requests.get(url)
    open(filename, "w").write(r.text)

In [ ]:
from openff.interchange import Interchange

from openff.toolkit import ForceField, Molecule

In [ ]:
ligand_molecule = Molecule("ligand.sdf")
sage = ForceField("openff-2.0.0.offxml")

ligand = Interchange.from_smirnoff(
    force_field=sage, topology=ligand_molecule.to_topology()
)

receptor_molecule = Molecule.from_polymer_pdb("receptor.pdb")

ff14sb = ForceField("ff14sb_off_impropers_0.0.2.offxml")
# Remove when https://github.com/openforcefield/amber-ff-porting/issues/37 resolved
ff14sb["Bonds"].fractional_bondorder_method = "AM1-Wiberg"
ff14sb.deregister_parameter_handler("ImproperTorsions")

receptor = Interchange.from_smirnoff(
    force_field=ff14sb, topology=receptor_molecule.to_topology()
)

complex_system = receptor + ligand

# TODO
# complex.box = pdbfile box vectors ...
# complex.positions = np.vstack([receptor.positions, ligand.positions])

### Export to OpenMM

In [ ]:
complex_system.to_openmm()

### Export to Amber

In [ ]:
# TODO: Fix inferring residue information with mixed topology
if False:
    complex_system.to_inpcrd("complex.inpcrd")
    complex_system.to_prmtop("complex.prmtop")

### Export to GROMACS

In [ ]:
# Broken but unclear why
if False:
    complex_system.to_gro("complex.gro")
    complex_system.to_top("complex.top")